In [247]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
from geopy.adapters import AioHTTPAdapter
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
#%load_ext autotime
import geopandas as gpd
import geopy
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm.notebook import tqdm_notebook
import reverse_geocoder as rg
import os

In [248]:
#usw1= pd.read_csv('../data/7890488/USC00042863.csv')

file_name = os.path.basename('USC00042863.csv')

usw1['station'] = file_name

usw1 = pd.DataFrame(usw1)

usw1

,Unnamed: 0,Date,tmax,tmin,prcp,station
0,1,1894-01-01,60.0,41.0,0.00,USC00042863.csv
1,2,1894-01-02,58.0,50.0,0.40,USC00042863.csv
2,3,1894-01-03,57.0,42.0,0.00,USC00042863.csv
3,4,1894-01-04,53.0,42.0,0.28,USC00042863.csv
4,5,1894-01-05,50.0,38.0,0.00,USC00042863.csv
...,...,...,...,...,...,...
46746,46747,2021-12-27,NaN,NaN,NaN,USC00042863.csv
46747,46748,2021-12-28,NaN,NaN,NaN,USC00042863.csv
46748,46749,2021-12-29,NaN,NaN,NaN,USC00042863.csv
46749,46750,2021-12-30,NaN,NaN,NaN,USC00042863.csv


In [249]:
# List all the CSV files in the directory
csv_files = [file for file in os.listdir('../data/7890488') if file.endswith('.csv')]

dfs = []

# Iterate over CSV files
for file in csv_files:
   
    df = pd.read_csv('../data/7890488/' + file)
    file_name = os.path.basename(file)
    
    df['station'] = file_name
    dfs.append(df)
uswd = pd.concat(dfs, ignore_index=True)

uswd

,Unnamed: 0,Date,tmax,tmin,prcp,station
0,1,1894-01-01,60.0,41.0,0.00,USC00042863.csv
1,2,1894-01-02,58.0,50.0,0.40,USC00042863.csv
2,3,1894-01-03,57.0,42.0,0.00,USC00042863.csv
3,4,1894-01-04,53.0,42.0,0.28,USC00042863.csv
4,5,1894-01-05,50.0,38.0,0.00,USC00042863.csv
...,...,...,...,...,...,...
10475909,46747,2021-12-27,32.0,1.2,0.22,USW00094973.csv
10475910,46748,2021-12-28,22.1,-5.8,0.06,USW00094973.csv
10475911,46749,2021-12-29,20.1,-4.9,0.00,USW00094973.csv
10475912,46750,2021-12-30,NaN,NaN,NaN,USW00094973.csv


In [250]:
#clean up file name column
uswd['station'] = uswd['station'].str.rstrip('.csv')
uswd

,Unnamed: 0,Date,tmax,tmin,prcp,station
0,1,1894-01-01,60.0,41.0,0.00,USC00042863
1,2,1894-01-02,58.0,50.0,0.40,USC00042863
2,3,1894-01-03,57.0,42.0,0.00,USC00042863
3,4,1894-01-04,53.0,42.0,0.28,USC00042863
4,5,1894-01-05,50.0,38.0,0.00,USC00042863
...,...,...,...,...,...,...
10475909,46747,2021-12-27,32.0,1.2,0.22,USW00094973
10475910,46748,2021-12-28,22.1,-5.8,0.06,USW00094973
10475911,46749,2021-12-29,20.1,-4.9,0.00,USW00094973
10475912,46750,2021-12-30,NaN,NaN,NaN,USW00094973


In [251]:
uswd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10475914 entries, 0 to 10475913
Data columns (total 6 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   Date        object 
 2   tmax        float64
 3   tmin        float64
 4   prcp        float64
 5   station     object 
dtypes: float64(3), int64(1), object(2)
memory usage: 479.5+ MB


In [252]:
#formatting city data to merge into weather
city = pd.read_csv('../data/city_info.csv')
city.drop(['Unnamed: 0', 'Name', 'Stn.Name', 'Stn.stDate', 'Stn.edDate'],axis=1, inplace=True)
city.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      461 non-null    object 
 1   Lat     461 non-null    float64
 2   Lon     461 non-null    float64
dtypes: float64(2), object(1)
memory usage: 10.9+ KB


In [253]:
lat = city.Lat
lon = city.Lon

result1 = []
for i in range(len(lat)):
     #Adding to result based on indexes
     result1.append((lat[i], lon[i]))

In [254]:
location_dic1 = rg.search(result1)

In [255]:
location_df1 = pd.DataFrame(location_dic1)
location_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   lat     461 non-null    object
 1   lon     461 non-null    object
 2   name    461 non-null    object
 3   admin1  461 non-null    object
 4   admin2  461 non-null    object
 5   cc      461 non-null    object
dtypes: object(6)
memory usage: 21.7+ KB


In [256]:
#formatting to prepare for join
location_df1['lat'] = location_df1['lat'].astype(float)
location_df1['lon'] = location_df1['lon'].astype(float)
city['lat'] = city['Lat']
city['lon'] = city['Lon']

In [257]:
#More formatting
location_df1['lat']= location_df1['lat'].round(decimals=3)
location_df1['lon']= location_df1['lon'].round(decimals=3)
city = city.round(decimals =3)

In [258]:
#joining for full station info
stations = city.join(location_df1, lsuffix = 'x')#, on = ['lat','lon'], how = 'inner')
stations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      461 non-null    object 
 1   Lat     461 non-null    float64
 2   Lon     461 non-null    float64
 3   latx    461 non-null    float64
 4   lonx    461 non-null    float64
 5   lat     461 non-null    float64
 6   lon     461 non-null    float64
 7   name    461 non-null    object 
 8   admin1  461 non-null    object 
 9   admin2  461 non-null    object 
 10  cc      461 non-null    object 
dtypes: float64(6), object(5)
memory usage: 39.7+ KB


In [260]:
#formatting to prepare for join to weather data
stations.drop(['Lat', 'Lon', 'latx', 'lonx', 'name', 'admin2', 'cc'],axis=1, inplace=True)

In [261]:
uswd.drop('Unnamed: 0', axis =1, inplace = True)

In [262]:
uswd['ID'] = uswd['station']

In [316]:
#merge into good dataframe!
stations.drop_duplicates(subset='ID', inplace=True)

weather = uswd.merge(stations, left_on='ID', right_on='ID')
#weather = uswd.join(stations, lsuffix = 'x', on = 'ID', how = 'inner')
#weather = uswd.merge(stations, left_on='ID', right_on='ID',
          #suffixes=('_left', '_right'))
weather.head(10)

,Date,tmax,tmin,prcp,station,ID,lat,lon,admin1
0,1894-01-01,60.0,41.0,0.00,USC00042863,USC00042863,33.119,-117.086,California
1,1894-01-02,58.0,50.0,0.40,USC00042863,USC00042863,33.119,-117.086,California
2,1894-01-03,57.0,42.0,0.00,USC00042863,USC00042863,33.119,-117.086,California
3,1894-01-04,53.0,42.0,0.28,USC00042863,USC00042863,33.119,-117.086,California
4,1894-01-05,50.0,38.0,0.00,USC00042863,USC00042863,33.119,-117.086,California
5,1894-01-06,50.0,32.0,0.00,USC00042863,USC00042863,33.119,-117.086,California
6,1894-01-07,53.0,30.0,0.00,USC00042863,USC00042863,33.119,-117.086,California
7,1894-01-08,56.0,33.0,0.00,USC00042863,USC00042863,33.119,-117.086,California
8,1894-01-09,61.0,36.0,0.00,USC00042863,USC00042863,33.119,-117.086,California
9,1894-01-10,61.0,34.0,0.00,USC00042863,USC00042863,33.119,-117.086,California


In [317]:
len(weather)

10475914

In [318]:
weather.isna().sum()

Date            0
tmax       583903
tmin       583010
prcp       525810
station         0
ID              0
lat             0
lon             0
admin1          0
dtype: int64

In [320]:
#add region column
# Create a dictionary mapping states to regions
region_mapping = {
    'Tennessee': 'SOUTH EASTERN',
    'North Carolina': 'SOUTH EASTERN',
    'South Carolina': 'SOUTH EASTERN',
    'Georgia': 'SOUTH EASTERN',
    'Aalabama': 'SOUTH EASTERN',
    'Mississippi': 'SOUTH EASTERN',
    'Florida': 'SOUTH EASTERN',
    'Kentucky': 'NORTH EASTERN',
    'Virgina': 'NORTH EASTERN',
    'West Virgina': 'NORTH EASTERN',
    'Indiana': 'NORTH EASTERN',
    'Ohio': 'NORTH EASTERN',
    'Michigan': 'NORTH EASTERN',
    'Pennsylvania': 'NORTH EASTERN',
    'DC': 'NORTH EASTERN',
    'Delaware': 'NORTH EASTERN',
    'New Jersey': 'NORTH EASTERN',
    'New York': 'NORTH EASTERN',
    'Connecticut': 'NORTH EASTERN',
    'Massachusetts': 'NORTH EASTERN',
    'Vermont': 'NORTH EASTERN',
    'New Hampshire': 'NORTH EASTERN',
    'Maine': 'NORTH EASTERN',
    'Rhode Island': 'NORTH EASTERN',
    'North Dakota': 'NORTH CENTRAL',
    'Minnesota': 'NORTH CENTRAL',
    'Wisconsin': 'NORTH CENTRAL',
    'South Dakota': 'NORTH CENTRAL',
    'Nebraska': 'NORTH CENTRAL',
    'Kansas': 'NORTH CENTRAL',
    'Missouri': 'NORTH CENTRAL',
    'Illinois': 'NORTH CENTRAL',
    'Iowa': 'NORTH CENTRAL',
    'Oklahoma': 'SOUTH CENTRAL',
    'Arkansas': 'SOUTH CENTRAL',
    'Louisiana': 'SOUTH CENTRAL',
    'Texas': 'SOUTH CENTRAL',
    'California': 'SOUTH WEST',
    'Nevada': 'SOUTH WEST',
    'Utah': 'SOUTH WEST',
    'Arizona': 'SOUTH WEST',
    'Colorado': 'SOUTH WEST',
    'New Mexico': 'SOUTH WEST',
    'Washington': 'NORTH WEST',
    'Oregon': 'NORTH WEST',
    'Idaho': 'NORTH WEST',
    'Montana': 'NORTH WEST',
    'Wyoming': 'NORTH WEST',


}

# Create a 'region' column based on the 'states' column
weather['region'] = weather['admin1'].map(region_mapping)

In [323]:
weather.to_csv('weather.csv', index = True)